This file computes optimal pricing given 50 clustered hotspots. 

Consider we have 100 more listings added to the market. This file computes the prices for these 100 listings and also adjusted prices for the existing lisitngs. 

When 100 more listings added to the market. The market prices are driven down by 17$ (21%)


In [ ]:
import zipfile
import csv
import json
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
import matplotlib.dates as mdates
from datetime import date
from sklearn.model_selection import train_test_split
import math

In [21]:
with zipfile.ZipFile('listings_mar2023_w_hotspots_and_50_clusters.zip', 'r') as zip_ref:
    with zip_ref.open('listings_mar2023_w_hotspots_and_50_clusters.csv', 'r') as file:
        df = pd.read_csv(file)
print(df.columns)
df.cluster_num.head()

Index(['Unnamed: 0', 'id', 'listing_url', 'scrape_id', 'last_scraped',
       'source', 'name', 'description', 'neighborhood_overview', 'picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights

0    13
1    35
2    44
3     4
4     4
Name: cluster_num, dtype: int64

In [19]:
df['price'] = df['price'].replace({r'\$':''},regex = True).replace({r',':''},regex = True).astype('float')
df1=df[df.price<800]

df2= df1[~df1.beds.isna()&~df1.reviews_per_month.isna()&~df1.review_scores_accuracy.isna()&~df1.review_scores_checkin.isna()
         &~df1.review_scores_communication.isna()&~df1.review_scores_location.isna()&~df1.review_scores_value.isna()]

In [23]:
# for top 50 hotspots

df2 = df2.copy() 
# create market share from reviews
df2.loc[:,'share'] =df2['reviews_per_month']/(df2.groupby('cluster_num')['reviews_per_month'].transform('sum')*3)

# 2/3 is arbitrary, won't affect result much, create mean valuation of the listing
df2['mean_value'] =np.log(df2['share']/(2/3))

The following code computes markup based on IV regression results.

In [30]:
df3=df2.copy()
from statsmodels.sandbox.regression.gmm import IV2SLS
# instrument for price endogeneity
df3.loc[:,'IV'] = df3.groupby(['cluster_num'])['id'].transform('count')

Y=df3.mean_value
#with amenties


license_dummy=1*(~df3.license.isna()) 
# Fill NAN by "missing" string
df3[['host_response_time']] = df3[['host_response_time']].fillna('Missing_response')
host_response_dummy=pd.get_dummies(df3.host_response_time)
#room_type dummy
room_type_dummy=pd.get_dummies(df3.room_type)
super_host_dummy =1*(df3.host_is_superhost=="t")

amentity_count=df3.amenities.str.split().str.len()
wifi = 1*df3.amenities.str.lower().str.contains('wifi', regex=True) 
washer = 1*df3.amenities.str.lower().str.contains('washer', regex=True) 
dryer = 1*df3.amenities.str.lower().str.replace("hair dryer", " ").str.lower().str.contains('dryer', regex=True)
hair_dryer = 1*df3.amenities.str.lower().str.contains('hair dryer', regex=True) 
free_parking = 1*df3.amenities.str.lower().str.contains('free parking', regex=True) 
AC = 1*df3.amenities.str.lower().str.contains('air conditioning', regex=True) 
TV= 1*df3.amenities.str.lower().str.contains('tv', regex=True) 
hottub = 1*df3.amenities.str.lower().str.contains('hot tub', regex=True) 
coffee = 1*df3.amenities.str.lower().str.contains('coffee', regex=True) 
microwave = 1*df3.amenities.str.lower().str.contains('microwave', regex=True) 
pool = 1*df3.amenities.str.lower().str.contains('pool', regex=True) 
shampoo = 1*df3.amenities.str.lower().str.contains('shampoo', regex=True) 


Y=df3.mean_value
X=pd.concat([df3.price,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1)
exog_constant=X
#sm.add_constant(X)
endog = Y
instr_constant = sm.add_constant(pd.concat([df3.IV,df3.beds,license_dummy,
             room_type_dummy,super_host_dummy,df3.review_scores_rating,amentity_count,wifi,
             washer,dryer,hair_dryer,free_parking,AC,TV,hottub,coffee,microwave,pool,shampoo],axis=1))
endog_results = IV2SLS(endog, exog_constant, instrument = instr_constant).fit()
endog_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          IV2SLS Regression Results                           
==============================================================================
Dep. Variable:             mean_value   R-squared:                      -9.988
Model:                         IV2SLS   Adj. R-squared:                 -9.996
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Mon, 29 May 2023                                         
Time:                        23:04:48                                         
No. Observations:               29053                                         
Df Residuals:                   29031                                         
Df Model:                          21                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
price                   -0.0516      0.003    -18.739      0.000      -0.057      -0.046
beds                     2.0003      0.104     19.310      0.000       1.797       2.203
license                  2.7092      0.155     17.445      0.000       2.405       3.014
Entire home/apt         -8.0975      0.429    -18.879      0.000      -8.938      -7.257
Hotel room              -8.0819      0.562    -14.389      0.000      -9.183      -6.981
Private room           -11.8268      0.435    -27.189      0.000     -12.679     -10.974
Shared room            -15.4286      0.579    -26.666      0.000     -16.563     -14.295
host_is_superhost        0.5075      0.070      7.209      0.000       0.370       0.645
review_scores_rating     0.3881      0.071      5.466      0.000       0.249       0.527
amenities                0.0269      0.001     18.348      0.000       0.024       0.030
amenities               -0.1481      0.294     -0.503      0.615      -0.725       0.429
amenities                1.1443      0.146      7.851      0.000       0.859       1.430
amenities               -1.2056      0.119    -10.110      0.000      -1.439      -0.972
amenities                0.2961      0.096      3.080      0.002       0.108       0.485
amenities                0.6600      0.072      9.144      0.000       0.519       0.801
amenities                0.6045      0.079      7.673      0.000       0.450       0.759
amenities                0.0533      0.108      0.496      0.620      -0.157       0.264
amenities                1.6854      0.144     11.739      0.000       1.404       1.967
amenities                0.4963      0.096      5.166      0.000       0.308       0.685
amenities               -1.2398      0.125     -9.893      0.000      -1.485      -0.994
amenities                0.4616      0.097      4.779      0.000       0.272       0.651
amenities                0.3929      0.091      4.317      0.000       0.215       0.571
==============================================================================
Omnibus:                     8872.781   Durbin-Watson:                   1.955
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39018.429
Skew:                           1.440   Prob(JB):                         0.00
Kurtosis:                       7.892   Cond. No.                     6.56e+03
==============================================================================
"""

In [ ]:
r = np.corrcoef(x, y)

In [31]:
#markup 
markup =[]
df3 = df3.copy()
# number of the markets (hotspots)
M = df3['cluster_num'].nunique()
Group_market = df3.groupby('cluster_num')
#elaciticies
ela = [] 
for num, i in enumerate(df3['cluster_num'].unique()):
    submarket = Group_market.get_group(i)
    share_m = np.array(submarket.share)
    dsdp_m =  endog_results.params['price']*(np.diag(share_m)-np.outer(share_m,share_m)) #price derivatives
    
    #ela
    ela_m =dsdp_m*np.array(np.array(submarket.price).reshape(-1, 1) )/np.array(submarket.share)# that's how i compute elasticities  
    ela.append(ela_m)
    
    # markup
    host_id = submarket.host_id.to_numpy()
    T = host_id [:, np.newaxis] == host_id 
    markup.append(-np.dot(np.linalg.inv(T*dsdp_m),share_m))
     # assign markup back to dataFrame df3
    df3.loc[df3['cluster_num'] == i, 'markup'] = markup[num]

In [32]:
df3.markup.head() #markup for the first 10 listing in the first market

#profit margain in percentage

df3['margin_perc'] = df3.markup/df3.price
print("average markup per listing is $",str(round(df3.markup.mean(),2)) )
print("average rate of return is",str(round(df3.margin_perc.mean(),2)))
df3['mc'] = df3['price']-df3['markup']

average markup per listing is $ 19.43
average rate of return is 0.18


In [33]:
## compute counterfactual price if a new listing added. 
# create 100 new fake listing which is the as the row 101-200
new_row = df3.iloc[101:200,:]

print(new_row)
df3.reset_index(drop=True, inplace=True)
new_row.reset_index(drop=True, inplace=True)
df4 = pd.concat([df3,new_row], axis=0,ignore_index=True)

     Unnamed: 0                  id  \
163         167            28809386   
164         168  783972360261400731   
166         170  676000420992053150   
167         171  652746661449522211   
168         172  738818475291675967   
..          ...                 ...   
292         298             8702595   
294         300  691935157380751426   
296         302             1153772   
297         303  752342881927585959   
298         304  574943161797157673   

                                         listing_url       scrape_id  \
163            https://www.airbnb.com/rooms/28809386  20230307175359   
164  https://www.airbnb.com/rooms/783972360261400731  20230307175359   
166  https://www.airbnb.com/rooms/676000420992053150  20230307175359   
167  https://www.airbnb.com/rooms/652746661449522211  20230307175359   
168  https://www.airbnb.com/rooms/738818475291675967  20230307175359   
..                                               ...             ...   
292             https://www

In [34]:
# compute the price 
tol = 1e-6


Group_market4 = df4.groupby('cluster_num')
for num, i in enumerate(df4['cluster_num'].unique()):
    submarket = Group_market4.get_group(i)
    new_price = np.array(submarket.price)
    diff=1
    while diff >tol: 
        mc= np.array(submarket.mc)
        new_share_m = np.array(submarket.mean_value)-endog_results.params['price']*(np.array(submarket.price)-new_price)
        dsdp_m = endog_results.params['price']*(np.diag(new_share_m)-np.outer(new_share_m,new_share_m)) #price derivatives

        #ela
        ela_m =dsdp_m*np.array(np.array(new_price).reshape(-1, 1) )/np.array(new_share_m)# that's how i compute elasticities  
        #ela.append(ela_m)

        # markup
        host_id = submarket.host_id.to_numpy()
        T = host_id [:, np.newaxis] == host_id 
        new_markup_m = -np.dot(np.linalg.inv(T*dsdp_m),new_share_m)
        old_price = new_price
        
        new_price =  mc+ new_markup_m
        diff= np.max(np.abs(old_price-new_price))
        #markup.append()
        
    # assign markup back to dataFrame df4
    df4.loc[df4['cluster_num'] == i, 'new markup'] = new_markup_m
    df4.loc[df4['cluster_num'] == i, 'new price'] = new_price


In [35]:
price_change =df4['price']-df4['new price']
print(price_change.mean())

price_change_perc =(df4['price']-df4['new price'])/df4['new price']
print(price_change_perc.mean())

17.9095916624234
0.2103738936033249


when 100 more listings added to the market. The market prices are driven down by 17$ (21%)